# Selection of Stocks / Distributed Processing #

In order to speed up the processsing we can try to distribute the load. Apache Spark is a popular framework for  large-scale data processing. 

However I gave it up because I was running into too many problems
- Jar Dependency hell because different compoentents were asking for different versions of Jackson
- The clustering does not work in Docker Swarm mode with OS/X
- It is not possible just to rely on Maven for the deployment in cluster mode.
- It needs to have a local installation if you want to start an instance in cluster mode

As a less painfull approach we show how to use the scripting-ws to achieve the same goal. This tool is easier to set up and deploy in a multihost environemnt.

In this document we demonstrate the different approaches and compare the runtimes.

## Setup ##
### Import Libraries ###

In [1]:
%classpath config resolver maven-public http://pschatzmann.ch:8081/repository/maven-public/
%classpath add mvn ch.pschatzmann:investor:0.9-SNAPSHOT
%classpath add mvn ch.pschatzmann:jupyter-jdk-extensions:0.0.1-SNAPSHOT
%classpath add mvn ch.pschatzmann:scripted-ws:0.0.1-SNAPSHOT
%classpath add mvn org.apache.commons:commons-collections4:4.1


Added new repo: maven-public


Added jars: [combinatoradix-0.8.2.jar, commons-jcs-core-2.0.jar, httpmime-4.5.2.jar, websocket-api-9.2.20.v20161216.jar, YahooFinanceAPI-3.12.3.jar, httpclient-4.5.3.jar, log4j-1.2.17.jar, xalan-2.7.2.jar, jcl-over-slf4j-1.7.25.jar, xml-apis-1.4.01.jar, slf4j-api-1.7.25.jar, htmlunit-core-js-2.23.jar, ant-1.5.jar, jackson-annotations-2.9.4.jar, timeseries-forecast-1.1.1.jar, slf4j-log4j12-1.7.25.jar, qdox-1.5.jar, sevenzipjbinding-all-platforms-9.20-2.00beta.jar, commons-codec-1.10.jar, cssparser-0.9.21.jar, jackson-core-2.9.4.jar, commons-cli-1.3.1.jar, jfreechart-1.5.0.jar, websocket-common-9.2.20.v20161216.jar, commons-io-2.5.jar, sojo-1.0.5.jar, jackson-databind-2.9.4.jar, ta4j-core-0.10.jar, neko-htmlunit-2.24.jar, junit-4.10.jar, websocket-client-9.2.20.v20161216.jar, jetty-util-9.2.20.v20161216.jar, commons-lang3-3.5.jar, investor-0.9-SNAPSHOT.jar, commons-attributes-api-2.2.jar, sac-1.3.jar, sevenzipjbinding-9.20-2.00beta.jar, jetty-io-9.2.20.v20161216.jar, xercesImpl-2.11.0.ja

Added jars: [concurrent-1.3.4.jar, jcs-1.3.jar, jupyter-jdk-extensions-0.0.1-SNAPSHOT.jar, logkit-1.0.1.jar, avalon-framework-4.1.3.jar, servlet-api-2.3.jar, in-memory-stardb-0.0.1-SNAPSHOT.jar, common-0.0.1-SNAPSHOT.jar, commons-logging-1.1.jar]


Added jars: [reactor-core-3.1.1.RELEASE.jar, javax.inject-2.5.0-b42.jar, reactive-streams-1.0.1.jar, scala-library-2.12.4.jar, commons-jcs-jcache-2.2.jar, commons-jcs-core-2.2.jar, objenesis-2.5.1.jar, jackson-dataformat-yaml-2.9.4.jar, netty-codec-4.1.20.Final.jar, jersey-container-jdk-http-2.26.jar, jackson-module-jaxb-annotations-2.9.4.jar, javax.annotation-api-1.2.jar, netty-transport-4.1.20.Final.jar, javax.inject-1.jar, hk2-utils-2.5.0-b42.jar, netty-common-4.1.20.Final.jar, netty-codec-dns-4.1.20.Final.jar, mimepull-1.9.6.jar, minlog-1.3.0.jar, netty-resolver-dns-4.1.20.Final.jar, byte-buddy-1.6.14.jar, scala-xml_2.12-1.0.6.jar, redisson-3.6.0.jar, javax.ws.rs-api-2.1.jar, cache-api-1.0.0.jar, snakeyaml-1.18.jar, commons-logging-1.2.jar, validation-api-1.1.0.Final.jar, jersey-server-2.26.jar, netty-handler-4.1.20.Final.jar, netty-resolver-4.1.20.Final.jar, scala-reflect-2.12.4.jar, hk2-api-2.5.0-b42.jar, jersey-hk2-2.26.jar, jersey-client-2.26.jar, osgi-resource-locator-1.0.1.ja

Added jar: [commons-collections4-4.1.jar]


### Imports ###

In [2]:
// our stock evaluation framwork
import ch.pschatzmann.dates._;
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.data.universe._;
import ch.pschatzmann.stocks.input._;
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.execution._;
import ch.pschatzmann.stocks.execution.fees._;
import ch.pschatzmann.stocks.execution.price._;
import ch.pschatzmann.stocks.parameters._;
import ch.pschatzmann.stocks.strategy._;
import ch.pschatzmann.stocks.strategy.optimization._;
import ch.pschatzmann.stocks.strategy.allocation._;
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.integration._;
import ch.pschatzmann.stocks.integration.ChartData.FieldName._;
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._;

// java
import java.util.stream.Collectors;
import java.util._;
import java.lang._;
import java.util.function.Consumer;

// log4j
import org.apache.log4j._;

// jupyter custom displayer
import ch.pschatzmann.display.Displayers

import ch.pschatzmann.dates._
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.input._
import ch.pschatzmann.stocks.accounting._
import ch.pschatzmann.stocks.accounting.kpi._
import ch.pschatzmann.stocks.execution._
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.execution.price._
import ch.pschatzmann.stocks.parameters._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.strategy.optimization._
import ch.pschatzmann.stocks.strategy.allocation._
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.integration._
import ch.pschatzmann.stocks.integration.ChartData.FieldName._
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._
import java.util.stream...

### Deactivate Logging and Caching ###

In [3]:
Displayers.setup()
Context.setCachingActive(false);
Context.isCachingActive();

false

## The Problem Statment ##
The following code provides the starting point for our investigation. It is a small scala logic which evaluates for all
Trading Strategies for all NASDAQ stocks.
We are comparing the runtimes of the different solution approaches with the help of our Timer functionality.

In [4]:
import scala.collection.JavaConverters
import java.util.Comparator


// this is our input data
var periods = Context.getDateRanges("2016-01-01","2017-01-01");
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
var strategies = TradingStrategyFactory.list()
var kpi = KPI.SharpeRatio
 
// Selection setup of best strategy for the defined period and KPI
var strategySelector = new StrategySelector(account, strategies, periods.get(0), kpi)


ch.pschatzmann.stocks.strategy.selection.StrategySelector@96a7757

In [5]:
var timer = new ch.pschatzmann.performance.Timer();

// and we limit the result to the best 50: TopNSet is thread save and synchronized 
var comparator = new StateComparator(kpi).asInstanceOf[Comparator[SelectionState]]
var top50 = new TopNSet[SelectionState](50, comparator)

// Now we can process the universe in par(allel)
var universe = JavaConverters.asScalaBufferConverter(new MarketUniverse("NASDAQ").list()).asScala.toSeq
universe.par.foreach( 
    stockID =>  top50.add(strategySelector.getMax(Context.getStockData(stockID)))
)

timer.getResult()

14 minutes 15 seconds

## Parallel Scala Processing ##
We can optimize the processing time by simply using a parallel collection. 

Note that we are collecting the result in our TopNSet which is implemented as a synchronized set whcih limits the number of entries. This is however an antipattern and not the recommended practice.

In [6]:
var timer = new ch.pschatzmann.performance.Timer();

var top50Parallel = new TopNSet[SelectionState](50, comparator)
var universeParallel = JavaConverters.asScalaBufferConverter(new MarketUniverse("NASDAQ").list()).asScala.toSeq.par
universeParallel.foreach( 
    stockID =>  top50Parallel.add(strategySelector.getMax(Context.getStockData(stockID)))
)

timer.getResult()

10 minutes 45 seconds

Here is the implementation which follows more the Scala recommendation:
- we calculate the result
- then we sort (in descending order with <) and take the first 5 elements
It has however the disadvantage that we need to sort the full result list only disregard all except the first 5 entries.

In [7]:
var timer = new ch.pschatzmann.performance.Timer();

var universeParallel = JavaConverters.asScalaBufferConverter(new MarketUniverse("NASDAQ").list()).asScala.toSeq.par
var result = universeParallel.map( stockID =>  strategySelector.getMax(Context.getStockData(stockID)))
var top5ViaSort = result.seq.sortWith{(leftE,rightE) => 
     leftE.getResult().getDouble(kpi) < rightE.getResult().getDouble(kpi)
}.reverse.take(5)

timer.getResult()

10 minutes 10 seconds

## Distributed Scripted Services ##
If we want to distribute the load to multiple machines with the help of e.g. Docker Swarm, we can use the scripted-ws framework. 
The functionality was made available with the following entry to the classpath:

- %classpath add mvn ch.pschatzmann scripted-ws 0.0.1-SNAPSHOT

This framwork has the folowing advantages 
- it can reuse existing functionality in an very easy way
- there are no separate compilation or deployment steps 
- it can be easily distributed (e.g. with Docker) in a multi machine environment with different operating systems and also works correcly if a worker is e.g. a OS/X machine. 


We create an instance of the class that we want to deploy as script and then we call the service method in a loop. In our case we  distribute the execution of the stategy selector. 

We check the connection to the services first:

In [8]:
import ch.pschatzmann.scripting._

var service = new ScriptingClient[SelectionState]("http://10.0.1.2:10001", "http://10.0.1.20:10001","http://10.0.1.33:10001","http://10.0.1.16:10001")
//var service = new ScriptingClient[SelectionState]( "http://10.0.1.33:10001")


ch.pschatzmann.scripting.errors.WebserviceException:  No active address available

Here is the implementation:

In [9]:
import ch.pschatzmann.scripting._
import scala.collection.JavaConverters

var timer = new ch.pschatzmann.performance.Timer();
// key data
var periods = Context.getDateRanges("2016-01-01","2017-01-01");
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
var strategies = TradingStrategyFactory.list()
var kpi = KPI.SharpeRatio

// setup distributed service object with the strategySelector
var dependencies = Dependency.create("stocks:stocks:1.3-SNAPSHOT:http://10.0.1.20:8081/repository/maven-public/")
var bindings = Bindings.create().value("strategySelector", new StrategySelector(account, strategies, periods.get(0), kpi))
var serviceDef = new Definition("js-strategySelector", null, bindings, "js",  dependencies)
var service = new ScriptingClient[SelectionState]( "http://10.0.1.20:10001","http://10.0.1.33:10001","http://10.0.1.16:10001")

service.register(serviceDef)

// this is the client method that we plan to call
var serviceMethod = "strategySelector.getMax(stockData)"

// Now we can process the universe in par(allel)
var universe = JavaConverters.asScalaBufferConverter(new MarketUniverse("NASDAQ").list()).asScala.toSeq
var comparator = new StateComparator(kpi).asInstanceOf[Comparator[SelectionState]]
var top50Dec = new TopNSet[SelectionState](50, comparator)
universe.par.foreach( 
    stockID => 
        top50Dec.add(
            service.call("js-strategySelector", serviceMethod, Bindings.create().value("stockData", Context.getStockData(stockID)), classOf[SelectionState])
        )
)
    
timer.getResult()

ch.pschatzmann.scripting.errors.WebserviceException:  No active address available

## Design Pattern : A Remote Executable Class ##
Now that we understand the mechanism we can improve the structure of our solution. In our Stocks Framework the StrategySelector is implementing the interface IStategySelector. So - in order to make a distributed implementation available we just need to implement a new class which extends from that interface.
As you can see all the concepts fall naturatly to their expected place: 
- The setup and registration of the service is in the constructor
- And the the getMax call is just forwarding the call to the service.

In [10]:
import ch.pschatzmann.stocks.strategy.selection.IStategySelector
import ch.pschatzmann.stocks.accounting._
import java.util._
import ch.pschatzmann.dates.DateRange
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.accounting.kpi.KPI
import ch.pschatzmann.stocks.accounting.kpi.KPI._
import ch.pschatzmann.scripting._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.execution.fees._;
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.accounting.kpi._;


class DistributedStrategySelector(account:IAccount, strategies:Collection[String], optimizationPeriod:DateRange, target:KPI) extends IStategySelector {
    var periods = Context.getDateRanges("2016-01-01","2017-01-01")
    var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
    var strategies = TradingStrategyFactory.list()
    var dependencies = Dependency.create("stocks:stocks:1.3-SNAPSHOT")
    var bindings = Bindings.create().value("strategySelector", new StrategySelector(account, strategies, periods.get(0), target))
    var serviceDef = new Definition("DistributedStrategySelector", null, bindings, "js",  new ArrayList())
    var service = new ScriptingClient[SelectionState]("http://10.0.1.20:10001","http://10.0.1.2:10001", "http://10.0.1.16:10001" )
    service.register(serviceDef)

    def getMax(stockData:IStockData):SelectionState = {
        var serviceMethod = "strategySelector.getMax(stockData)"
        return service.call("DistributedStrategySelector", serviceMethod, Bindings.create().value("stockData", stockData), classOf[SelectionState])
    }
    
    def getOptimizationParameter():KPI = {
        return target
    }
}


import ch.pschatzmann.stocks.strategy.selection.IStategySelector
import ch.pschatzmann.stocks.accounting._
import java.util._
import ch.pschatzmann.dates.DateRange
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.accounting.kpi.KPI
import ch.pschatzmann.stocks.accounting.kpi.KPI._
import ch.pschatzmann.scripting._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.accounting.kpi._
defined class DistributedStrategySelector


With this we have a class we can use the same logic that we have used in chapter "4) Selection" by just replacing the
StrategySelector with our new DistributedStrategySelector

In [11]:
var timer = new ch.pschatzmann.performance.Timer();
    
var kpi = KPI.SharpeRatio
var periods = Context.getDateRanges("2016-01-01","2017-01-01");
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
var file = new java.io.File("NASDAQ-2017-01-Distributed.json")
if (!file.exists()) {
    var universe = new MarketUniverse("NASDAQ")
    var strategies = TradingStrategyFactory.list()
    var strategySelector = new DistributedStrategySelector(account, strategies, periods.get(0), KPI.SharpeRatio)
    var stockSelector = new StockSelector(strategySelector, new Restartable("restart-v1S.ser",50))
    // calculate the result
    var result = stockSelector.getSelection(200, universe, new MarketArchiveHttpReader())
    // save the result to a file
    result.save(file) 
} 

// provide result even if we did not run the selection above
var resultS = new SelectionResult()
resultS.load(file)
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader);
var executor = new StrategyExecutor(trader, allocationStrategy);
executor.addStrategy(resultS.getStrategies(new MarketArchiveHttpReader()));
executor.run(periods.get(1));

timer.getResult()

3 seconds

## Using Java Multithreading With A Threadpool ##

Each webservice is staring a thread on the server, therfore it is of advantage to keep the server busy by executing many calls in parallel. This can be achieved and managed with the help of a Java ThreadPool. We start 10 Threads which in our example are thitting 3 physical servers: So we execute constently 3 requests for each server.

The framework automatically limits the number of open server requests in order to avoid that the server is overloaded.

In [12]:
import ch.pschatzmann.scripting._
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.data.universe._
import scala.collection.JavaConverters
import java.util.concurrent._
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.strategy._;
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.scripting.utils.ThrottlingLoadBalancer

class SelectStrategyCallable(service:ScriptingClient[SelectionState], stockID:IStockID) extends Callable[SelectionState] {
    override def call():SelectionState = {
        var serviceMethod = "strategySelector.getMax(stockData)"
        service.call("js-strategySelector", serviceMethod, Bindings.create().value("stockData", Context.getStockData(stockID)), classOf[SelectionState])       
    }
}

object StrategyServiceWithThreadPool {
    var kpi = KPI.SharpeRatio
    var service = new ScriptingClient[SelectionState]( "http://10.0.1.20:10001","http://10.0.1.2:10001","http://10.0.1.16:10001")
    service.setSerializer(classOf[ch.pschatzmann.scripting.serialization.JavaSerializer]);
    var loadBalancer = new ThrottlingLoadBalancer()
    // define custom load balancer
    loadBalancer.setMaxOpenRequests(10)
    service.setLoadBalancer(loadBalancer)
    
    def registerService() {
        var periods = Context.getDateRanges("2016-01-01","2017-01-01");
        var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
        var strategies = TradingStrategyFactory.list()
        var dependencies = Dependency.create("stocks:stocks:1.3-SNAPSHOT:http://10.0.1.20:8081/repository/maven-public/")
        var bindings = Bindings.create().value("strategySelector", new StrategySelector(account, strategies, periods.get(0), kpi))
        var serviceDef = new Definition("js-strategySelector", null, bindings, "js",  dependencies)
        service.register(serviceDef)  
    }
    
    def process():Seq[SelectionState] = {
        val numberOfThreads = 9
        val topLimit = 50
        // Now we can process the universe in n parallel threads
        var universe = JavaConverters.asScalaBufferConverter(new MarketUniverse("NASDAQ").list()).asScala.toSeq
        val pool = Executors.newFixedThreadPool(numberOfThreads) 
        val futures = universe.map(stockID => pool.submit(new SelectStrategyCallable(service, stockID)))
        val result = futures.map(future => future.get()).filter(result => result!=null)
        val top5 = result.sortWith{(leftE,rightE) =>  leftE.getResult().getDouble(kpi) < rightE.getResult().getDouble(kpi)}.reverse.take(topLimit)           
        top5
    }
}

var timer = new ch.pschatzmann.performance.Timer();

StrategyServiceWithThreadPool.registerService
var result = StrategyServiceWithThreadPool.process

timer.getResult()

ch.pschatzmann.scripting.errors.WebserviceException:  No active address available

Here are some timings:

- 5  Threads/20 max open requests -> 17 minutes 15 seconds
- 5  Threads/10 max open requests -> 10 minutes 4 seconds
- 6  Threads/10 max open requests -> 9 minutes 39 seconds
- 9  Threads/10 max open requests -> 9 minutes 39 seconds
- 10 Threads/20 max open requests -> 10 minutes 55 seconds
- 10 Threads/30 max open requests -> 11 minutes 2 seconds
- 50 Threads/20 max open requests -> 23 minutes 55 seconds

This is not very impressing! So we look how we can improve the timings:


## Minimizing the Number of Calls by Producig Batches of Data #

For each method call there is quite a big overhead because we need to set up the scripting engine, go thru the serialization of paramters and on top of this we have the network overhead for each call.

So it is much better to give the server a bigger chunk of work. If you remember our last tuturial we used a StockSelector on a list of stocks. So it is a better strategy to split this list, calculate the result decentrally and then consolidate the result:

In [13]:
new MarketUniverse("NASDAQ").list().size()

3683

So we have roughly 3600 entries to distribute. So we partion by 800 which gives roughly 5 "batches" which need to be processed

In [14]:
import ch.pschatzmann.scripting._
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.parameters._;
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.strategy._;
import org.apache.commons.collections4.ListUtils
import ch.pschatzmann.stocks.input.MarketArchiveHttpReader
import java.util.concurrent._
import java.util._
import scala.collection.JavaConverters
import ch.pschatzmann.scripting.utils.ThrottlingLoadBalancer


class SelectStockCallable(service:ScriptingClient[TopNSet[SelectionState]], stockList: ArrayList[IStockID], topLimit:Integer) extends Callable[TopNSet[SelectionState]] {

    override def call():TopNSet[SelectionState] = {
        var serviceMethod = "stockSelector.getSelection(topLimit, stockList, reader).getResult()"
        var bindings = Bindings.create()
            .value("topLimit", topLimit)
            .value("stockList", stockList)
            .value("reader", new MarketArchiveHttpReader())
        var result = service.call("js-stockSelector", serviceMethod, bindings, classOf[List[SelectionState]])   
        return result.asInstanceOf[TopNSet[SelectionState]]
    }                                
}
                     
 object StocksServiceWithThreadPool {
    var kpi = KPI.SharpeRatio
    var service = new ScriptingClient[TopNSet[SelectionState]]("http://10.0.1.2:10001", "http://10.0.1.20:10001","http://10.0.1.16:10001")
    //var service = new ScriptingClient[TopNSet[SelectionState]]("http://10.0.1.2:10001", "http://10.0.1.20:10001","http://10.0.1.33:10001","http://10.0.1.16:10001")
    //var service = new ScriptingClient[TopNSet[SelectionState]]( "http://10.0.1.16:10001", "http://10.0.1.33:10001")
    //service.setSerializer(classOf[ch.pschatzmann.scripting.serialization.JavaSerializer]);
    var loadBalancer = new ThrottlingLoadBalancer()
    // define custom load balancer
    loadBalancer.setMaxOpenRequests(20)
    service.setLoadBalancer(loadBalancer)
    val topLimit = 50
    val partitionSize = 150
    val numberOfThreads = 8

    def registerService() = {
        var periods = Context.getDateRanges("2016-01-01","2017-01-01");
        var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
        var strategies = TradingStrategyFactory.list()
        var strategySelector = new StrategySelector(account, strategies, periods.get(0), kpi)
        var bindings = Bindings.create()
            .value("stockSelector", new StockSelector(strategySelector))
        var dependencies = Dependency.create("stocks:stocks:1.3-SNAPSHOT:http://10.0.1.20:8081/repository/maven-public/")
        var serviceDef = new Definition("js-stockSelector", null, bindings, "js",  dependencies)
        service.register(serviceDef)
    }
    
    def process():TopNSet[SelectionState] = {
        var universeList = new MarketUniverse("NASDAQ").list()
        var listOfListOfStocks = ListUtils.partition(universeList, partitionSize)
        var scalaListOfListOfStocks = JavaConverters.asScalaBufferConverter(listOfListOfStocks).asScala.toSeq
        val pool = Executors.newFixedThreadPool(numberOfThreads) 
        val futures = scalaListOfListOfStocks.map(listOfStocks => pool.submit(new SelectStockCallable(service, new ArrayList(listOfStocks), topLimit)))
        var comparator = new StateComparator(kpi).asInstanceOf[Comparator[SelectionState]]
        var top50X = new TopNSet[SelectionState](topLimit, comparator)                     
        val result = futures.map(futures => futures.get())
        result.foreach(list => top50X.addAll(list))
        top50X
    }
}
                    
                                     
var timer = new ch.pschatzmann.performance.Timer()

StocksServiceWithThreadPool.registerService
var result = StocksServiceWithThreadPool.process

timer.getResult()

ch.pschatzmann.scripting.errors.WebserviceException:  No active address available

- Size/Th/Open
- 300 / 6 / 10 -> 9 minutes 9 seconds
- 200 / 6 / 10 -> 6 minutes 33 seconds
- 200 / 6 / 20 -> 6 minutes 33 seconds
- 200 / 9 / 10 -> 7 minutes 48 seconds
- 100 / 6 / 10 -> 7 minutes 48 seconds

## Example w/o Java Threds ##

In [15]:
import ch.pschatzmann.scripting._
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.parameters._;
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.strategy._;
import org.apache.commons.collections4.ListUtils
import ch.pschatzmann.stocks.input.MarketArchiveHttpReader
import java.util.concurrent._
import java.util._
import scala.collection.JavaConverters
import ch.pschatzmann.scripting.utils.ThrottlingLoadBalancer

                     
 object StocksService {
    var kpi = KPI.SharpeRatio
    var service = new ScriptingClient[TopNSet[SelectionState]]( "http://10.0.1.20:10001","http://10.0.1.33:10001","http://10.0.1.16:10001")
    var loadBalancer = new ThrottlingLoadBalancer()
    // define custom load balancer
    loadBalancer.setMaxOpenRequests(10)
    service.setLoadBalancer(loadBalancer)
    val topLimit = 50
    val partitionSize = 200

    def registerService() = {
        var periods = Context.getDateRanges("2016-01-01","2017-01-01");
        var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(100.00))
        var strategies = TradingStrategyFactory.list()
        var strategySelector = new StrategySelector(account, strategies, periods.get(0), kpi)
        var bindings = Bindings.create()
            .value("stockSelector", new StockSelector(strategySelector))
        var dependencies = Dependency.create("stocks:stocks:1.3-SNAPSHOT:http://10.0.1.20:8081/repository/maven-public/")
        var serviceDef = new Definition("js-stockSelector", null, bindings, "js",  dependencies)
        service.register(serviceDef)
    }
     
     def processListOfStocks(stockList: List[IStockID]):TopNSet[SelectionState] = {
        var serviceMethod = "stockSelector.getSelection(topLimit, stockList, reader).getResult()"
        var bindings = Bindings.create()
            .value("topLimit", topLimit)
            .value("stockList", stockList)
            .value("reader", new MarketArchiveHttpReader())
        var result = service.call("js-stockSelector", serviceMethod, bindings, classOf[List[SelectionState]])   
        return result.asInstanceOf[TopNSet[SelectionState]]
    }       
    
    def processAll():TopNSet[SelectionState] = {
        var universeList = new MarketUniverse("NASDAQ").list()
        var listOfListOfStocks = ListUtils.partition(universeList, partitionSize)
        var scalaListOfListOfStocks = JavaConverters.asScalaBufferConverter(listOfListOfStocks).asScala.toSeq
        var comparator = new StateComparator(kpi).asInstanceOf[Comparator[SelectionState]]
        var top50Parallel = new TopNSet[SelectionState](50, comparator)

        scalaListOfListOfStocks.par.foreach(
            list => top50Parallel.addAll(processListOfStocks(new ArrayList(list)))
        )
        top50Parallel
    }
}
                    
                                     
var timer = new ch.pschatzmann.performance.Timer()

StocksService.registerService
var result = StocksService.processAll

timer.getResult()

ch.pschatzmann.scripting.errors.WebserviceException:  No active address available